# 한글 형태소 분석기 - SOYNLP

- pip install soynlp
- !wget https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt -O 2016-10-20.txt
- 위는 말뭉치 다운로드
- Noun Extractor 말뭉치 데이터로부터 명사를 끄집어내는 코드. 


Noun Extractor
명사 추출을 하기 위해 여러 시도를 한 결과, v1, news, v2 세 가지 버전이 만들어졌습니다. 가장 좋은 성능을 보이는 것은 v2 입니다.

WordExtractor 는 통계를 이용하여 단어의 경계 점수를 학습하는 것일 뿐, 각 단어의 품사를 판단하지는 못합니다. 때로는 각 단어의 품사를 알아야 하는 경우가 있습니다. 또한 다른 품사보다도 명사에서 새로운 단어가 가장 많이 만들어집니다. 명사의 오른쪽에는 -은, -는, -라는, -하는 처럼 특정 글자들이 자주 등장합니다. 문서의 어절 (띄어쓰기 기준 유닛)에서 왼쪽에 위치한 substring 의 오른쪽에 어떤 글자들이 등장하는지 분포를 살펴보면 명사인지 아닌지 판단할 수 있습니다. soynlp 에서는 두 가지 종류의 명사 추출기를 제공합니다. 둘 모두 개발 단계이기 때문에 어떤 것이 더 우수하다 말하기는 어렵습니다만, NewsNounExtractor 가 좀 더 많은 기능을 포함하고 있습니다. 추후, 명사 추출기는 하나의 클래스로 정리될 예정입니다.

In [2]:
import soynlp
soynlp.__version__

'0.0.493'

In [3]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="data/2016-10-20.txt")

('data/2016-10-20.txt', <http.client.HTTPMessage at 0x269b83fee20>)

In [ ]:
# 위가 말뭉치 어마어마하게 튼 . 가보면 저런 데이터들이 주욱 있어. 


- soynlp에서 사용될 단어 만들기

In [4]:
# 훈련 데이터를 다수의 문서로 분리
from soynlp import DoublespaceLineCorpus
corpus = DoublespaceLineCorpus("data/2016-10-20.txt")
len(corpus)


30091

In [5]:
# 훈련과정 
from soynlp.word import WordExtractor
word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 0.753 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


- 소이 엔엘피를 이용한 단어 토큰화
- 우리는 형태소 분석하고 명사 추출하려고 해 
- 한글로 하는것은 # 형태소 분석, # 품사 부착 # 명사 추출 세가지 정도 작업 하기 위한 과정에 
소이 엔엘피 형태소 분석기중 엘 토크나이저 사용 .
- 사용법 스코어스에 

In [6]:
from soynlp.tokenizer import LTokenizer
# word:score.cohesion_forward 워드와 스코어 적어줘. 이 것들을 아규먼트로 사용할것. 

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

In [7]:
import joblib

joblib.dump(scores, 'data/scores.pkl')

['data/scores.pkl']

In [8]:
from soynlp.noun import LRNounExtractor_v2

noun_extractor = LRNounExtractor_v2(verbose=True)
nouns = noun_extractor.train_extract(corpus)
# 할때 마다 시간 걸려 / 추출한 것은 파일로 만들어 둘것. 

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 403896 from 30091 sents. mem=0.822 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=4434442, mem=1.454 Gb
[Noun Extractor] batch prediction was completed for 119705 words
[Noun Extractor] checked compounds. discovered 70639 compounds
[Noun Extractor] postprocessing detaching_features : 109312 -> 92205
[Noun Extractor] postprocessing ignore_features : 92205 -> 91999
[Noun Extractor] postprocessing ignore_NJ : 91999 -> 90643
[Noun Extractor] 90643 nouns (70639 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.575 Gb                    
[Noun Extractor] 76.63 % eojeols are covered


In [ ]:
# import joblib
# scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
# joblib.dump(scores, 'data/scores.pkl')

In [9]:
noun_scores = {noun:score[0] for noun, score in nouns.items() if len(noun) > 1}
joblib.dump(noun_scores, 'data/noun_scores.pkl')

['data/noun_scores.pkl']

In [ ]:
# 네이버 지식인 크롤링 햇던 데이터 셀레늄과 지식인 썻던. 